# <img align="left" src="./images/movie_camera.png"     style=" width:40px;  " > Practice lab: Collaborative Filtering Recommender Systems

En este ejercicio, implementará el filtrado colaborativo para construir un sistema de recomendación de películas. 

# <img align="left" src="./images/film_reel.png"     style=" width:40px;  " > Outline
- [ 1 - Notation](#1)
- [ 2 - Recommender Systems](#2)
- [ 3 - Movie ratings dataset](#3)
- [ 4 - Collaborative filtering learning algorithm](#4)
  - [ 4.1 Collaborative filtering cost function](#4.1)
    - [ Exercise 1](#ex01)
- [ 5 - Learning movie recommendations](#5)
- [ 6 - Recommendations](#6)
- [ 7 - Congratulations!](#7)


##  Packages <img align="left" src="./images/film_strip_vertical.png"     style=" width:40px;   " >
Utilizaremos los ya conocidos paquetes NumPy y Tensorflow.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from recsys_utils import *

|General <br />  Notation  | Description| Python (if any) |
|:-------------|:------------------------------------------------------------||
| $r(i,j)$     | escalar; = 1 si el usuario j calificó la película i = 0 en caso contrario             ||
| $y(i,j)$     | escalar; = calificación dada por el usuario j en la película i (si r(i,j) = 1 está definido) ||
|$\mathbf{w}^{(j)}$ | vector; parámetros para el usuario j ||
|$b^{(j)}$     |  escalar; parámetro para el usuario j ||
| $\mathbf{x}^{(i)}$ |   vector; clasificación de las características de la película i       ||     
| $n_u$        | número de usuarios |num_users|
| $n_m$        | número de películas | num_movies |
| $n$          | número de películas | num_features                    |
| $\mathbf{X}$ |  matriz de vectores $\mathbf{x}^{(i)}$         | X |
| $\mathbf{W}$ |  matrix of vectors $\mathbf{w}^{(j)}$         | W |
| $\mathbf{b}$ |  vector de parámetros de sesgo $b^{(j)}$ | b |
| $\mathbf{R}$ | matriz de elementos $r(i,j)$                    | R |




<a name="2"></a>
## 2 - Sistemas de recomendación <img align="left" src="./images/film_rating.png" style=" width:40px; " >
En este laboratorio, implementará el algoritmo de aprendizaje de filtrado colaborativo y lo aplicará a un conjunto de datos de calificaciones de películas.
El objetivo de un sistema de recomendación por filtrado colaborativo es generar dos vectores: Para cada usuario, un "vector de parámetros" que encarna los gustos de las películas de un usuario. Para cada película, un vector de características del mismo tamaño que encarna alguna descripción de la película. El producto punto de los dos vectores más el término de sesgo debería producir una estimación de la calificación que el usuario podría dar a esa película.

El siguiente diagrama detalla cómo se aprenden estos vectores.

<figure>
   <img src="./images/ColabFilterLearn.PNG"  style="width:740px;height:250px;" >
</figure>

Las calificaciones existentes se proporcionan en forma de matriz como se muestra. $Y$ contiene las calificaciones; de 0,5 a 5 inclusive en pasos de 0,5. 0 si la película no ha sido calificada. $R$ tiene un 1 cuando las películas han sido calificadas. Las películas están en filas, los usuarios en columnas. Cada usuario tiene un vector de parámetros $w^{user}$ y un sesgo. Cada película tiene un vector de características $x^{película}$. Estos vectores se aprenden simultáneamente utilizando las valoraciones existentes de usuarios/películas como datos de entrenamiento. Un ejemplo de entrenamiento se muestra arriba: $\mathbf{w}^{(1)} \cdot \mathbf{x}^{(1)} + b^{(1)} = 4$. Cabe destacar que el vector de características $x^{película}$ debe satisfacer a todos los usuarios, mientras que el vector de usuarios $w^{usuario}$ debe satisfacer a todas las películas. Este es el origen del nombre de este enfoque: todos los usuarios colaboran para generar el conjunto de valoraciones. 

<figure>
   <img src="./images/ColabFilterUse.PNG"  style="width:640px;height:250px;" >
</figure>

Una vez aprendidos los vectores de características y los parámetros, pueden utilizarse para predecir cómo podría calificar un usuario una película no calificada. Esto se muestra en el diagrama anterior. La ecuación es un ejemplo de predicción de la calificación del usuario uno en la película cero.


En este ejercicio, implementará la función `cofiCostFunc` que calcula la función objetivo del filtrado colaborativo
objetivo del filtrado colaborativo. Después de implementar la función objetivo, utilizarás un bucle de entrenamiento personalizado de TensorFlow para aprender los parámetros del filtrado colaborativo. El primer paso es detallar el conjunto de datos y las estructuras de datos que se utilizarán en el laboratorio.

<a name="3"></a>
## 3 - Conjunto de datos de clasificación de películas <img align="left" src="./images/film_rating.png" style=" width:40px; " >
El conjunto de datos se deriva del conjunto de datos [MovieLens "ml-latest-small"](https://grouplens.org/datasets/movielens/latest/).   
[F. Maxwell Harper y Joseph A. Konstan. 2015. Los conjuntos de datos de MovieLens: Historia y contexto. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1-19:19. <https://doi.org/10.1145/2827872>]

El conjunto de datos original tiene 9000 películas calificadas por 600 usuarios. Se ha reducido el tamaño del conjunto de datos para centrarse en las películas de los años posteriores al 2000. Este conjunto de datos consiste en calificaciones en una escala de 0,5 a 5 en incrementos de 0,5 pasos. El conjunto de datos reducido tiene $n_u = 443$ usuarios, y $n_m= 4778$ películas. 

A continuación, se cargará el conjunto de datos de películas en las variables $Y$ y $R$.

La matriz $Y$ (una matriz $n_m \times n_u$) almacena las valoraciones $y^{(i,j)}$. La matriz $R$ es una matriz indicadora de valor binario, en la que $R(i,j) = 1$ si el usuario $j$ ha dado una calificación a la película $i$, y $R(i,j)=0$ en caso contrario. 

A lo largo de esta parte del ejercicio, también se trabajará con las matrices
matrices, $\mathbf{X}$, $\mathbf{W}$ y $\mathbf{b}$: 

$$\mathbf{X} = 
\begin{bmatrix}
--- (\mathbf{x}^{(0)})^T --- \\
--- (\mathbf{x}^{(1)})^T --- \\
\vdots \\
--- (\mathbf{x}^{(n_m-1)})^T --- \\
\end{bmatrix} , \quad
\mathbf{W} = 
\begin{bmatrix}
--- (\mathbf{w}^{(0)})^T --- \\
--- (\mathbf{w}^{(1)})^T --- \\
\vdots \\
--- (\mathbf{w}^{(n_u-1)})^T --- \\
\end{bmatrix},\quad
\mathbf{ b} = 
\begin{bmatrix}
 b^{(0)}  \\
 b^{(1)} \\
\vdots \\
b^{(n_u-1)} \\
\end{bmatrix}\quad
$$ 

La fila $i$-ésima de $\mathbf{X}$ corresponde al
vector de características $x^{(i)}$ para la $i$-ésima película, y la $j$-ésima fila de
$\mathbf{W}$ corresponde a un vector de parámetros $\mathbf{w}^{(j)}$, para el
usuario $j$-ésimo. Tanto $x^{(i)}$ como $\mathbf{w}^{(j)}$ son vectores $n$-dimensionales
$n$. Para los fines de este ejercicio, se utilizará $n=10$, y
por tanto, $\mathbf{x}^{(i)}$ y $\mathbf{w}^{(j)}$ tienen 10 elementos.
En consecuencia, $\mathbf{X}$ es un
$n_m \ ~ 10$ matriz y $\mathbf{W}$ es un $n_u \ ~ 10$ matriz.

Comenzaremos cargando el conjunto de datos de clasificación de películas para entender la estructura de los datos.
Cargaremos $Y$ y $R$ con el conjunto de datos de películas.  
También cargaremos $\mathbf{X}$, $\mathbf{W}$, y $\mathbf{b}$ con valores precalculados. Estos valores se aprenderán más tarde en el laboratorio, pero vamos a utilizar los valores precalculados para desarrollar el modelo de costes.

In [2]:
#Load data
X, W, b, num_movies, num_features, num_users = load_precalc_params_small()
Y, R = load_ratings_small()

print("Y", Y.shape, "R", R.shape)
print("X", X.shape)
print("W", W.shape)
print("b", b.shape)
print("num_features", num_features)
print("num_movies",   num_movies)
print("num_users",    num_users)

Y (4778, 443) R (4778, 443)
X (4778, 10)
W (443, 10)
b (1, 443)
num_features 10
num_movies 4778
num_users 443


In [3]:
#  A partir de la matriz, podemos calcular estadísticas como la valoración media.
tsmean =  np.mean(Y[0, R[0, :].astype(bool)])
print(f"Average rating for movie 1 : {tsmean:0.3f} / 5" )

Average rating for movie 1 : 3.400 / 5


<a name="4"></a>
## 4 - Algoritmo de aprendizaje de filtrado colaborativo <img align="left" src="./images/film_filter.png" style=" width:40px; " >

Ahora, comenzarás a implementar el algoritmo de aprendizaje de filtrado colaborativo
de filtrado colaborativo. Empezarás por implementar la función objetivo. 

El algoritmo de filtrado colaborativo en el entorno de las recomendaciones de películas
considera un conjunto de vectores de parámetros $n$-dimensionales
$\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)}$, $\mathbf{w}^{(0)},...,\mathbf{w}^{(n_u-1)}$ y $b^{(0)},...,b^{(n_u-1)}$, donde el modelo predice la calificación de la película $i$ por el usuario $j$ as
$y^{(i,j)} = \mathbf{w}^{(j)}\cdot \mathbf{x}^{(i)} + b^{(j)}$ . Dado un conjunto de datos que consiste en
un conjunto de valoraciones producidas por algunos usuarios sobre algunas películas, se desea
aprender los vectores de parámetros $\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},
\mathbf{w}^{(0)},...,\mathbf{w}^{(n_u-1)}$  y $b^{(0)},...,b^{(n_u-1)}$ que produce el mejor ajuste (minimiza
el error al cuadrado).

Completará el código en cofiCostFunc para calcular la función de coste
para el filtrado colaborativo.

<a name="4.1"></a>
### 4.1 Función de coste del filtrado colaborativo

La función de coste del filtrado colaborativo viene dada por

$$J({\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},\mathbf{w}^{(0)},b^{(0)},...,\mathbf{w}^{(n_u-1)},b^{(n_u-1)}})= \frac{1}{2}\sum_{(i,j):r(i,j)=1}(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2
+\underbrace{
\frac{\lambda}{2}
\sum_{j=0}^{n_u-1}\sum_{k=0}^{n-1}(\mathbf{w}^{(j)}_k)^2
+ \frac{\lambda}{2}\sum_{i=0}^{n_m-1}\sum_{k=0}^{n-1}(\mathbf{x}_k^{(i)})^2
}_{regularization}
\tag{1}$$

El primer sumatorio en (1) es "para todo $i$, $j$ donde $r(i,j)$ es igual a $1$" y podría escribirse:

$$
= \frac{1}{2}\sum_{j=0}^{n_u-1} \sum_{i=0}^{n_m-1}r(i,j)*(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2
+\text{regularization}
$$

Ahora debe escribir cofiCostFunc (función de coste de filtrado colaborativo) para devolver este coste.

<a name="ex01"></a>
### Ejercicio 1

**Implementación del bucle for:**   
Comience por implementar la función de coste utilizando bucles for.
Considere desarrollar la función de coste en dos pasos. Primero, desarrolle la función de coste sin regularización. Un caso de prueba que no incluye la regularización se proporciona a continuación para probar su implementación. Una vez que funcione, añada la regularización y ejecute las pruebas que incluyen la regularización.  Tenga en cuenta que debería acumular el coste para el usuario $j$ y la película $i$ sólo si $R(i,j) = 1$.

In [4]:
# GRADED FUNCTION: cofi_cost_func
# UNQ_C1

def cofi_cost_func(X, W, b, Y, R, lambda_):
    """
    Devuelve el coste del filtrado por contenido
    Args:
      X (ndarray (num_movies,num_features)): matriz de características de los elementos
      W (ndarray (num_users,num_features)) : matriz de parámetros de usuario
      b (ndarray (1, num_users) : vector de parámetros de usuario
      Y (ndarray (num_movies,num_users) : matriz de valoraciones de películas por parte de los usuarios
      R (ndarray (num_movies,num_users) : matriz, donde R(i, j) = 1 si la i-ésima película fue calificada por el j-ésimo usuario
      lambda_ (float): parámetro de regularización
    Devuelve:
      J (float) : Coste
    """
    nm, nu = Y.shape
    J = 0
    ### START CODE HERE ###  
    #  1/2 ∑𝑗=0𝑛𝑢−1∑𝑖=0𝑛𝑚−1𝑟(𝑖,𝑗)∗(𝐰(𝑗)⋅𝐱(𝑖)+𝑏(𝑗)−𝑦(𝑖,𝑗))2
    for j in range(nu):
        
        w   = W[j,:]
        b_j = b[0,j]
        
        for i in range(nm):
        
            x = X[i,:]
            y = Y[i,j]
            r = R[i,j]
            J += np.square(r * (np.dot(w,x) + b_j - y ) )
    J = J/2
    J += (lambda_/2) * (np.sum(np.square(W)) + np.sum(np.square(X)))

    ### END CODE HERE ### 
    
    return J

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    You can structure the code in two for loops similar to the summation in (1).   
    Implement the code without regularization first.   
    Note that some of the elements in (1) are vectors. Use np.dot(). You can also use np.square().
    Pay close attention to which elements are indexed by i and which are indexed by j. Don't forget to divide by two.
    
```python     
    ### START CODE HERE ###  
    for j in range(nu):
        
        
        for i in range(nm):
            
            
    ### END CODE HERE ### 
```    
<details>
    <summary><font size="2" color="darkblue"><b> Click for more hints</b></font></summary>
        
    Here is some more details. The code below pulls out each element from the matrix before using it. 
    One could also reference the matrix directly.  
    This code does not contain regularization.
    
```python 
    nm,nu = Y.shape
    J = 0
    ### START CODE HERE ###  
    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = 
            y = 
            r =
            J += 
    J = J/2
    ### END CODE HERE ### 

```
    
<details>
    <summary><font size="2" color="darkblue"><b>Last Resort (full non-regularized implementation)</b></font></summary>
    
```python 
    nm,nu = Y.shape
    J = 0
    ### START CODE HERE ###  
    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = X[i,:]
            y = Y[i,j]
            r = R[i,j]
            J += np.square(r * (np.dot(w,x) + b_j - y ) )
    J = J/2
    ### END CODE HERE ### 
```
    
<details>
    <summary><font size="2" color="darkblue"><b>regularization</b></font></summary>
     Regularization just squares each element of the W array and X array and them sums all the squared elements.
     You can utilize np.square() and np.sum().

<details>
    <summary><font size="2" color="darkblue"><b>regularization details</b></font></summary>
    
```python 
    J += (lambda_/2) * (np.sum(np.square(W)) + np.sum(np.square(X)))
```
    
</details>
</details>
</details>
</details>

    


In [5]:
# Reducir el tamaño del conjunto de datos para que se ejecute más rápido
num_users_r = 4
num_movies_r = 5 
num_features_r = 3

X_r = X[:num_movies_r, :num_features_r]
W_r = W[:num_users_r,  :num_features_r]
b_r = b[0, :num_users_r].reshape(1,-1)
Y_r = Y[:num_movies_r, :num_users_r]
R_r = R[:num_movies_r, :num_users_r]

# Evaluate cost function
J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost: {J:0.2f}")

Cost: 13.67


In [6]:
# Evaluate cost function with regularization 
J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

Cost (with regularization): 28.09


**Expected Output**:

28.09

In [7]:
# Public tests
from public_tests import *
test_cofi_cost_func(cofi_cost_func)

All tests passed!


**Implementación vectorizada**

Es importante crear una implementación vectorizada para calcular $J$, ya que luego será llamada muchas veces durante la optimización. El álgebra lineal utilizada no es el objetivo de esta serie, por lo que se proporciona la implementación. Si usted es un experto en álgebra lineal, siéntase libre de crear su versión sin hacer referencia al código de abajo. 

Ejecute el código de abajo y verifique que produce los mismos resultados que la versión no vectorizada.

In [8]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    """
    Devuelve el coste del filtrado basado en el contenido
    Vectorizado para la velocidad. Utiliza operaciones de tensorflow para ser compatible con el bucle de entrenamiento personalizado.
    Args:
      X (ndarray (num_movies,num_features)): matriz de características de los elementos
      W (ndarray (num_users,num_features)) : matriz de parámetros de usuario
      b (ndarray (1, num_users) : vector de parámetros de usuario
      Y (ndarray (num_movies,num_users) : matriz de valoraciones de películas por parte de los usuarios
      R (ndarray (num_movies,num_users) : matriz, donde R(i, j) = 1 si la i-ésima película fue calificada por el j-ésimo usuario
      lambda_ (float): parámetro de regularización
    Devuelve:
      J (float) : Coste
    """
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y) * R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [9]:
# Evaluate cost function
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost: {J:0.2f}")

# Evaluate cost function with regularization 
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

Cost: 13.67
Cost (with regularization): 28.09


**Expected Output**:  
Cost: 13.67  
Cost (with regularization): 28.09

<a name="5"></a>
## 5 - Recomendaciones de películas de aprendizaje <img align="left" src="./images/film_man_action.png" style=" width:40px; " >
------------------------------

Una vez que hayas terminado de implementar la función de costes de filtrado colaborativo
, puedes empezar a entrenar tu algoritmo para hacer
recomendaciones de películas para ti. 

En la celda de abajo, puedes introducir tus propias opciones de películas. El algoritmo hará recomendaciones por ti. Hemos rellenado algunos valores de acuerdo con nuestras preferencias, pero después de que tengas las cosas funcionando con nuestras elecciones, deberías cambiar esto para que coincida con tus gustos.
La lista de todas las películas del conjunto de datos se encuentra en el archivo [movie list](data/small_movie_list.csv).

In [10]:
movieList, movieList_df = load_Movie_List_pd()

my_ratings = np.zeros(num_movies)          #  Initialize my ratings

# Comprueba el archivo small_movie_list.csv para ver el ID de cada película en nuestro conjunto de datos
# Por ejemplo, Toy Story 3 (2010) tiene el ID 2700, por lo que para calificarla con "5", se puede establecer
my_ratings[2700] = 5 

#Or suppose you did not enjoy Persuasion (2007), you can set
my_ratings[2609] = 2;

# We have selected a few movies we liked / did not like and the ratings we
# gave are as follows:
my_ratings[929]  = 5 # El Señor de los Anillos: El Retorno del Rey
my_ratings[246]  = 5    # Shrek (2001)
my_ratings[2716] = 3    # Inception
my_ratings[1150] = 5    # Incredibles, The (2004)
my_ratings[382]  = 2    # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[366]  = 5    # Harry Potter y la piedra filosofal (2001)
my_ratings[622]  = 5    # Harry Potter y la cámara secreta (2002)
my_ratings[988]  = 3    # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[2925] = 1    # Louis Theroux: Law & Disorder (2008)
my_ratings[2937] = 1    # Nada que declarar (Rien à déclarer)
my_ratings[793]  = 5    # Piratas del Caribe: La maldición de la Perla Negra (2003))
my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

print('\nNew user ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0 :
        print(f'Rated {my_ratings[i]} for  {movieList_df.loc[i,"title"]}');


New user ratings:

Rated 5.0 for  Shrek (2001)
Rated 5.0 for  Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Rated 2.0 for  Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Rated 5.0 for  Harry Potter and the Chamber of Secrets (2002)
Rated 5.0 for  Pirates of the Caribbean: The Curse of the Black Pearl (2003)
Rated 5.0 for  Lord of the Rings: The Return of the King, The (2003)
Rated 3.0 for  Eternal Sunshine of the Spotless Mind (2004)
Rated 5.0 for  Incredibles, The (2004)
Rated 2.0 for  Persuasion (2007)
Rated 5.0 for  Toy Story 3 (2010)
Rated 3.0 for  Inception (2010)
Rated 1.0 for  Louis Theroux: Law & Disorder (2008)
Rated 1.0 for  Nothing to Declare (Rien à déclarer) (2010)


Ahora, añadamos estas críticas a $Y$ y $R$ y normalicemos las valoraciones.

In [11]:
# Reload ratings
Y, R = load_ratings_small()

# Add new user ratings to Y
Y = np.c_[my_ratings, Y]

# Add new user indicator matrix to R
R = np.c_[(my_ratings != 0).astype(int), R]

# Normalize the Dataset
Ynorm, Ymean = normalizeRatings(Y, R)

Preparemos el entrenamiento del modelo. Inicialice los parámetros y seleccione el optimizador Adam.

In [12]:
#  Useful Values
num_movies, num_users = Y.shape
num_features = 100

# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

Las operaciones implicadas en el aprendizaje simultáneo de $w$, $b$ y $x$ no entran en las típicas "capas" que ofrece el paquete de redes neuronales TensorFlow.  En consecuencia, el flujo utilizado en el curso 2: Model, Compile(), Fit(), Predict(), no es directamente aplicable. En su lugar, podemos utilizar un bucle de entrenamiento personalizado.

Recordemos de los laboratorios anteriores los pasos del descenso de gradiente.
- repetir hasta la convergencia:
    - calcular el paso hacia adelante
    - calcular las derivadas de la pérdida relativa a los parámetros
    - actualizar los parámetros utilizando la tasa de aprendizaje y las derivadas calculadas 
    
TensorFlow tiene la maravillosa capacidad de calcular las derivadas por ti. Esto se muestra a continuación. Dentro de la sección `tf.GradientTape()`, las operaciones sobre las variables de Tensorflow son rastreadas. Cuando se llama a `tape.gradient()` posteriormente, devolverá el gradiente de la pérdida relativa a las variables rastreadas. Los gradientes pueden entonces ser aplicados a los parámetros usando un optimizador. 
Esta es una introducción muy breve a una característica útil de TensorFlow y otros marcos de aprendizaje automático. Se puede encontrar más información investigando los "bucles de entrenamiento personalizados" dentro del marco de interés.

In [14]:
iterations = 200
lambda_ = 1
for iter in range(iterations):
    # Utilizar la cinta de gradiente de TensorFlow
    # para registrar las operaciones utilizadas para calcular el coste 
    with tf.GradientTape() as tape:

        # Calcule el coste (el pase de ida está incluido en el coste)
        cost_value = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)

    # Utilizar la cinta de gradientes para recuperar automáticamente
    # los gradientes de las variables entrenables con respecto a la pérdida
    grads = tape.gradient( cost_value, [X,W,b] )

    # Ejecuta un paso de descenso de gradiente actualizando
    # el valor de las variables para minimizar la pérdida.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Registrar periódicamente.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 2321191.3
Training loss at iteration 20: 136168.7
Training loss at iteration 40: 51863.3
Training loss at iteration 60: 24598.8
Training loss at iteration 80: 13630.4
Training loss at iteration 100: 8487.6
Training loss at iteration 120: 5807.7
Training loss at iteration 140: 4311.6
Training loss at iteration 160: 3435.2
Training loss at iteration 180: 2902.1


<a name="6"></a>
## 6 - Recomendaciones
A continuación, calculamos las valoraciones de todas las películas y usuarios y mostramos las películas recomendadas. Estas se basan en las películas y valoraciones introducidas como `mi_valoración[]` más arriba. Para predecir la calificación de la película $i$ para el usuario $j$, se calcula $\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)}$. Esto puede calcularse para todas las valoraciones utilizando la multiplicación de matrices.

In [15]:
# Hacer una predicción utilizando pesos y sesgos entrenados
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

#restablecer la media
pm = p + Ymean

my_predictions = pm[:,0]

# #Predicciones de ordenación
ix = tf.argsort(my_predictions, direction='DESCENDING')

for i in range(17):
    j = ix[i]
    if j not in my_rated:
        print(f'Predicting rating {my_predictions[j]:0.2f} for movie {movieList[j]}')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f'Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {movieList[i]}')

Predicting rating 4.49 for movie My Sassy Girl (Yeopgijeogin geunyeo) (2001)
Predicting rating 4.48 for movie Martin Lawrence Live: Runteldat (2002)
Predicting rating 4.48 for movie Memento (2000)
Predicting rating 4.47 for movie Delirium (2014)
Predicting rating 4.47 for movie Laggies (2014)
Predicting rating 4.47 for movie One I Love, The (2014)
Predicting rating 4.46 for movie Particle Fever (2013)
Predicting rating 4.45 for movie Eichmann (2007)
Predicting rating 4.45 for movie Battle Royale 2: Requiem (Batoru rowaiaru II: Chinkonka) (2003)
Predicting rating 4.45 for movie Into the Abyss (2011)


Original vs Predicted ratings:

Original 5.0, Predicted 4.90 for Shrek (2001)
Original 5.0, Predicted 4.84 for Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Original 2.0, Predicted 2.13 for Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Original 5.0, Predicted 4.88 for Harry Potter and the Chamber of Secrets (2002)
Original 5.0, Predic

En la práctica, se puede utilizar información adicional para mejorar nuestras predicciones. Arriba, las valoraciones predichas para los primeros cientos de películas se encuentran en un rango pequeño. Podemos aumentar lo anterior seleccionando entre esas películas principales, las películas que tienen valoraciones medias altas y las películas con más de 20 valoraciones. Esta sección utiliza un marco de datos [Pandas](https://pandas.pydata.org/) que tiene muchas características de clasificación útiles.

In [16]:
filter=(movieList_df["number of ratings"] > 20)
movieList_df["pred"] = my_predictions
movieList_df = movieList_df.reindex(columns=["pred", "mean rating", "number of ratings", "title"])
movieList_df.loc[ix[:300]].loc[filter].sort_values("mean rating", ascending=False)

,pred,mean rating,number of ratings,title
1743,4.030965,4.252336,107,"Departed, The (2006)"
2112,3.985287,4.238255,149,"Dark Knight, The (2008)"
211,4.477792,4.122642,159,Memento (2000)
929,4.887053,4.118919,185,"Lord of the Rings: The Return of the King, The..."
2700,4.796530,4.109091,55,Toy Story 3 (2010)
653,4.357304,4.021277,188,"Lord of the Rings: The Two Towers, The (2002)"
1122,4.004469,4.006494,77,Shaun of the Dead (2004)
1841,3.980647,4.000000,61,Hot Fuzz (2007)
3083,4.084633,3.993421,76,"Dark Knight Rises, The (2012)"
2804,4.434171,3.989362,47,Harry Potter and the Deathly Hallows: Part 1 (...


<a name="7"></a>
## 7 - Congratulations! <img align="left" src="./images/film_award.png"     style=" width:40px;  " >
You have implemented a useful recommender system!